In [41]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math

# auto-encoder network architecture

In [42]:
def autoencoder(dimensions=[278, 140, 70, 30]):
    """
    Build a stacked deep autoencoder with tied weights, that is w = wT.

    return a dict.
    
    Parameters
    ----------
    dimensions : list, optional
        The number of neurons for each layer of the autoencoder.
    Returns
    -------
    x : Tensor
        Input placeholder to the network
    z : Tensor
        Inner-most latent representation
    y : Tensor
        Output reconstruction of the input
    loss : Tensor
        Overall cost to use for training
    """    
    # input to the network
    x = tf.placeholder(tf.float32, [None, dimensions[0]], name='x')
    current_input = x
    print ('current_input', current_input.shape)
    #---------------------------
    # Build the encoder
    #---------------------------
    encoder = [] # for putting the weight of encoder, w1,w2,..
    for layer_i, n_output in enumerate(dimensions[1:]):
        print ('layer_i-encoder', layer_i)
        print ('n_output-encoder', n_output)
        n_input = int(current_input.get_shape()[1]) # [0]: batch_szie, [1]:input_dim
        print ('n_input', n_input)
        W = tf.Variable(
            tf.random_uniform([n_input, n_output],
                              minval = -1.0 / math.sqrt(n_input),
                              maxval = 1.0 / math.sqrt(n_input)))
        b = tf.Variable(tf.zeros([n_output]))
        # saving layer of encoding for decoder
        encoder.append(W)
        output = tf.nn.tanh(tf.matmul(current_input, W) + b)
        # assign current_input
        current_input = output
    #---------------------------
    # latent representation (output of encoder)
    #---------------------------
    z = current_input
    encoder.reverse() # [...,w2,w1]
    
    #---------------------------
    # Build the decoder using the same weights
    #---------------------------
    for layer_i, n_output in enumerate(dimensions[:-1][::-1]):
        print ('layer_i-decoder', layer_i)
        print ('n_output-decoder', n_output)
        W = tf.transpose(encoder[layer_i])
        b = tf.Variable(tf.zeros([n_output]))
        output = tf.nn.tanh(tf.matmul(current_input, W) + b)
        # assign current_input
        current_input = output

    # now have the reconstruction through the network
    y = current_input
    # Define loss and, minimize the mean squared error
    loss = tf.reduce_mean(tf.pow(x - y, 2)) 

    return {'x': x, 'z': z, 'y': y, 'loss': loss}


# input

In [43]:
# reload again for filling
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_comoress = ['index','TARGET', 'SK_ID_CURR']
df.drop(no_need_to_comoress, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)
print (df.shape)


(356251, 281)
(356251, 278)


# raw_feature_generator
### raw_feature_generator which allows to quickly set up Python generators that can automatically turn image files on disk into batches of pre-processed tensors. This is what we will use here.

In [44]:
length = len(df)
print ('length',length)
idx = np.arange(length) # 1-D array
print ('idx', idx.shape, idx)


length 356251
idx (356251,) [     0      1      2 ... 356248 356249 356250]


In [45]:
def raw_feature_generator(batch_size = 128, shuffle = True, num_epochs = 10000, allow_smaller_final_batch = False):
    epoch_num = 0
    while epoch_num < num_epochs:
        print ('epoch_num : {}'.format(epoch_num))
        if shuffle:
            np.random.shuffle(idx)
        for i in range(0, length, batch_size):
            batch_idx = idx[i: i + batch_size]
            if not allow_smaller_final_batch and len(batch_idx) != batch_size:
                break # terminate the loop
            yield df.values[batch_idx]
        epoch_num += 1
    

# training 

In [46]:
# display_step
display_step = 1000
# learning_rate
learning_rate = 0.001
# define auto-encoder network architecture
ae = autoencoder(dimensions=[278, 140, 70, 30])
# optimizer
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(ae['loss'])

# Initialize the variables (i.e. assign their default value)
# We create a session to use the graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Training
batch_size = 128
n_epochs = 10000
for batch_xs in raw_feature_generator(batch_size = 128, num_epochs = 10000):
    print ('batch_xs', batch_xs.shape)
    #-------------------------
    # feature scaling: make different feature have the same scaling
    #-------------------------
    #train = np.array([img - mean_img for img in batch_xs])
    # train
    _, loss = sess.run([optimizer, ae['loss']], feed_dict={ae['x']: batch_xs})
    if i % display_step == 0 or i == 1:
        print('Step %i: Minibatch Loss: %f' % (i, loss))
Z

current_input (?, 278)
layer_i-encoder 0
n_output-encoder 140
n_input 278
layer_i-encoder 1
n_output-encoder 70
n_input 140
layer_i-encoder 2
n_output-encoder 30
n_input 70
layer_i-decoder 0
n_output-decoder 70
layer_i-decoder 1
n_output-decoder 140
layer_i-decoder 2
n_output-decoder 278
epoch_num : 0
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 0: Minibatch Loss: nan
batch_xs (128, 278)
Step 

KeyboardInterrupt: 